In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import soundfile as sf
import scipy.signal as signal
import matplotlib.pyplot as plt # to support librosa display
import seaborn as sns
import IPython.display as ipd # for playing audio
from collections import Counter
from tqdm import tqdm
import gc
import os
import sys
import shutil
import random

from joblib import Parallel, delayed
from functools import partial

import librosa # audio proccessing
import librosa.display # cool audio visuals
from librosa import feature as lf



from skimage.io import imread
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, accuracy_score
from collections import Counter
import glob
import pickle


import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, GaussianNoise, Reshape, Lambda, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Activation, LSTM, GRU, Bidirectional
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Normalization
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
from tensorflow.keras.models import load_model

np.random.seed(42)
tf.random.set_seed(100)

In [ ]:
data_tp=pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_tp.csv')
data_fp=pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_fp.csv')

In [ ]:
data_tp.head(10)

In [ ]:
data_fp.head(10)

In [ ]:

train = "../input/output-files/MFCC/train"
test = "../input/output-files/MFCC/test"
train_files = os.listdir(train)
test_files = os.listdir(test) 
output_dir = "./"

In [ ]:
os.listdir("/kaggle")

In [ ]:
len(os.listdir("/kaggle/input/output-files/MFCC/test"))

In [ ]:
len(os.listdir("/kaggle/input/output-files/MFCC/train"))

In [ ]:
all_arrays_train = []
for npfile in train_files:
    all_arrays_train.append(np.load(os.path.join(train, npfile)))
all_arrays_train = np.array(all_arrays_train)
np.save('train_all.npy',all_arrays_train)

In [ ]:
train_mat= np.load('train_all.npy')
train_mat.ndim 



In [ ]:
train_mat.shape

In [ ]:
all_arrays_test = []
for npfile in test_files:
    all_arrays_test.append(np.load(os.path.join(test, npfile)))
all_arrays_train = np.array(all_arrays_test)
np.save('test_all.npy',all_arrays_test)

In [ ]:
test_mat=np.load('test_all.npy')
test_mat.ndim 

In [ ]:
test_mat.shape

In [ ]:
df1 = data_tp.iloc[:, 0:2]
rec= data_tp['recording_id'].values
y_labels = data_tp['species_id'].values
y = to_categorical(y_labels, num_classes=24)
print (data_tp.shape, y.shape)
df1


In [ ]:
X=np.empty((0,76, 143, 1),int)
labels=[]
for i,j,k in df1.itertuples():
    data = np.load('../input/output-files/MFCC/train/'+j+'.npy')
    data = np.expand_dims(data,axis=0)
    X=np.append(X,data,axis=0)
    labels.append(k)
    
 

labels=np.array(labels)
labels=labels.astype(int)

print(labels.shape)
Y=np.zeros((labels.size,24))

Y[np.arange(labels.size),labels] = 1


print(X.shape)
print(Y.shape)

In [ ]:
class_weight = compute_class_weight('balanced',np.arange(24), y_labels)
class_weights = dict(zip(np.arange(24), class_weight))
class_weights

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2,
                                                      stratify = y_labels, shuffle = True,
                                                      random_state = 42)
#x_train = x_train[..., np.newaxis]
#x_valid = x_valid[..., np.newaxis]
print (x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

# CNN Model only

In [ ]:

def create_model(input_shape = (76,143,1)):
  model = Sequential()
  model.add(Input(shape = input_shape))
  model.add(Resizing(200,200))
  
  model.add(Conv2D(32, (3,3), kernel_initializer = 'he_normal', padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  


  model.add(Conv2D(64, kernel_size = (3,3),  padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(128, kernel_size = (3,3), padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(256, kernel_size = (3,3), padding = 'same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Dropout(0.2))


  model.add(Flatten())

  model.add(Dense(512, kernel_initializer = 'he_normal'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.3))

  model.add(Dense(24, activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['acc'])
  model.summary()
  return model

In [ ]:

model = create_model()

In [ ]:

model_checkpoint = os.path.join(train,'models','tp_model.hdf5')

cp = ModelCheckpoint(model_checkpoint, monitor='val_loss', verbose=1, save_best_only=True)

lr = ReduceLROnPlateau(
      monitor='val_loss', factor=0.3, patience=10, verbose=0,
      mode='auto'
)

callbacks = [cp, lr]
history = model.fit(x_train, y_train, validation_data = (x_valid, y_valid),
                    epochs = 50, batch_size = 32, callbacks = callbacks, 
                    class_weight = class_weights)

In [ ]:
def plot_history(history):
    fig,axs = plt.subplots(1,2, figsize = (15,6))
    axs[0].plot(history.history['acc'])
    axs[0].plot(history.history['val_acc'])
    axs[0].set_title('model accuracy')
    axs[0].set_ylabel('accuracy')
    axs[0].set_xlabel('epoch')
    axs[0].legend(['train', 'val'], loc='upper left')
    
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set_title('model loss')
    axs[1].set_ylabel('loss')
    axs[1].set_xlabel('epoch')
    axs[1].legend(['train', 'val'], loc='upper right')
    
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
model.evaluate(x_valid, y_valid)

In [ ]:
y_pred = np.argmax(model.predict(x_valid),axis = 1)
y_true = np.argmax(y_valid, axis = 1)
print (classification_report(y_true,y_pred ))

# Model with both CNN and LSTM

In [ ]:
#Input shape (76,143,1) if first extracting feature by conv else (76,143) if first layer is RNN layer
def create_model(input_shape = (76,143)):
  model = Sequential()
  model.add(Input(input_shape))
  model.add(Normalization())
  
  model.add(Bidirectional(LSTM(90, kernel_initializer = 'he_normal', return_sequences= True)))
  model.add(BatchNormalization())


  model.add(Lambda(lambda x:tf.expand_dims(x, axis = -1)))
  
  model.add(Resizing(200,200))
  
  model.add(Conv2D(64, (3,3), kernel_initializer = 'he_normal', padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  


  model.add(Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Dropout(0.2))


  model.add(Conv2D(256, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same'))
  #model.add(GaussianNoise(0.3))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Dropout(0.2))

  '''model.add(Conv2D(256, (1,1)))
  model.add(Conv2D(32,(1,1)))
  model.add(Conv2D(1,(1,1)))
  
  model.add(Reshape((12,12)))

  model.add(Bidirectional(LSTM(units = 12, return_sequences=True)))
  #model.add(LSTM(12))
  '''
  model.add(Flatten())

  model.add(Dense(512, kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.3))

  model.add(Dense(24, kernel_initializer = 'he_normal', activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['acc'])
  model.summary()
  return model

In [ ]:

model = create_model()

In [ ]:
model_checkpoint = os.path.join(train,'models','tp_model.hdf5')

cp = ModelCheckpoint(model_checkpoint, monitor='val_loss', verbose=1, save_best_only=True)

lr = ReduceLROnPlateau(
      monitor='val_loss', factor=0.3, patience=10, verbose=0,
      mode='auto'
)

callbacks = [cp, lr]

history = model.fit(x_train, y_train, validation_data = (x_valid, y_valid),
                    epochs = 70, batch_size = 32, callbacks = callbacks, 
                    class_weight = class_weights)